In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn.preprocessing import MultiLabelBinarizer
import json

In [ ]:
def extract_names(json_string):
    try:
        data = json.loads(json_string)

        return [item.get("name") for item in data if "name" in item]
    except Exception as e:
        return []


In [ ]:
movies_df = pd.read_csv("tmdb_5000_movies.csv")
# movies_df.original_language.unique()
# drop non-English movies
# movies_df = movies_df[movies_df.original_language == "en"]
# movies_df.to_csv("tmdb_5000_movies.csv", index=False)


In [ ]:
credits_df = pd.read_csv("../../../../Downloads/tmdb_5000_credits.csv/tmdb_5000_credits.csv")
#credits_df = pd.read_csv("tmdb_5000_credits.csv")
# for all indices in movies_df, drop the rows for ids not in credits_df
#credits_df = credits_df[credits_df['movie_id'].isin(movies_df['id'])]
#credits_df.to_csv("tmdb_5000_credits.csv", index=False)

top5_cast = list()
for i in credits_df['cast']:
    movie_actors = list()
    try:
        # get top 5 actors
        fish = json.loads(i)
        for j in range(5):
            #print((fish[j]['name']))
            movie_actors.append(fish[j]['name'])
      
    except:
        #print(i)
        pass
    top5_cast.append(movie_actors)

for i in top5_cast:
    if len(i) < 5:
        print(i)

only_5 = [i for i in top5_cast if len(i) == 5]
print(len(only_5))
# there are only 4354 movies with 5 actors listed
edited_credits_df = credits_df[credits_df.index.isin([i for i in range(len(top5_cast)) if len(top5_cast[i]) == 5])]


In [ ]:
combined_df = movies_df.merge(right = credits_df, left_on = "id",right_on = "movie_id", how = "inner")
combined_df

In [ ]:
combined_df = combined_df.replace({"[]": float("nan")})

In [ ]:
combined_df.isnull().sum()

In [ ]:
combined_df.columns

In [ ]:
# movies_df[movies_df.isnull()["release_date"]]
combined_df = combined_df.drop(columns=["homepage", "overview", "tagline", "status", "title_x", "title_y", "movie_id",]).dropna().reset_index(drop=True)
combined_df

In [ ]:
combined_df.head(1)

In [ ]:
combined_df["genres"] = combined_df["genres"].apply(extract_names)
combined_df["keywords"] = combined_df["keywords"].apply(extract_names)
combined_df["production_companies"] = combined_df["production_companies"].apply(extract_names)
combined_df["production_countries"] = combined_df["production_countries"].apply(extract_names)
combined_df["spoken_languages"] = combined_df["spoken_languages"].apply(extract_names)
combined_df["cast"] = combined_df["cast"].apply(extract_names)
# combined_df["crew"] = combined_df["crew"].apply(extract_names)

combined_df["cast"] = combined_df["cast"].map(lambda x: x[:5].copy())
combined_df.head()

In [ ]:
combined_df.dtypes

In [ ]:
combined_df.original_language.value_counts()

In [ ]:
len(combined_df)

In [ ]:
combined_df[combined_df.revenue == 0][["budget", "original_title"]]

In [ ]:
combined_df.status.value_counts()

In [ ]:
# get the directors from the crew column from credits_df
def extract_directors(data_frame):

    directors = list()
    for i in data_frame['crew']:
        d_list = list()
        sub_string = '"job": "Director"'
        count = i.count(sub_string)
        try:
            for j in range(count):
                start_index = i.index(sub_string)
                # get the substring from the start_index to the next closing curly brace
                end_index = i.index("}", start_index)
                director_substring = i[start_index-30:end_index+1]
                # extract the name value from the substring
                name_start_index = director_substring.index('"name": "') + len('"name": "')
                name_end_index = director_substring.index('"', name_start_index)
                director_name = director_substring[name_start_index:name_end_index]
                d_list.append(director_name)
                # remove the processed substring from i
                i = i[end_index+1:]
        except:
            #print(i)
            pass
        directors.append(d_list)
        
    directors = [i for i in directors if len(i) == 1]
    return directors

# num of movies with one director for all english movies, and only those with 5 actors listed, respectively.
print(len(extract_directors(credits_df)))
print(len(extract_directors(edited_credits_df)))

4197
4089
